Лабораторная работа №10. ОСНОВЫ ОБРАБОТКИ ЕСТЕСТВЕННОГО ЯЗЫКА (NLP). ЗАДАЧА ТЕМАТИЧЕСКОГО МОДЕЛИРОВАНИЯ

ЗАДАНИЕ
1. Для выполнения задания используйте датасет с данными о спаме (https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset).
2. Самостоятельно реализовать BoW, TF-IDF.
3. Решить задачу классификации с понижением размерности.
 Использовать самостоятельно реализованные модели из предыдущих ЛР.
4. Решить задачу тематического моделирования с помощью LDA.

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
os.chdir('../data')
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [3]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df = df.drop('Unnamed: 2', axis=1)

In [5]:
df = df.drop('Unnamed: 3', axis=1)

In [6]:
df = df.drop('Unnamed: 4', axis=1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [8]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [9]:
X = df['v2']
y = df['v1']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Самостоятельно реализовать BoW, TF-IDF.

In [11]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [12]:
# Создание экземпляра CountVectorizer
vectorizer = CountVectorizer()

# Преобразование текстовых данных в матрицу признаков
X_bow = vectorizer.fit_transform(X)

In [13]:
# Разделение на тренировочный и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X_bow, y, test_size=0.2, random_state=42)

In [14]:
# Обучение и оценка модели
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9787234042553191


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Создание экземпляра TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Преобразование текстовых данных в матрицу признаков
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Разделение на тренировочный и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [16]:
# Обучение и оценка модели
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9555125725338491


In [12]:
def bag_of_words(corpus):
    # создается множество unique_words, которое содержит все уникальные слова, встречающиеся в документах
    unique_words = set(word for doc in corpus for word in doc.split())
    # создается словарь, в котором каждому уникальному слову присваивается уникальный индекс
    # enumerate возвращает индекс и значение каждого элемента в объекте
    word_dict = {word: i for i, word in enumerate(unique_words)}
    # создается пустой список bow_corpus, который впоследствии будет заполнен векторами BoW для каждого документа
    bow_corpus = []
    # цикл проходит через каждый документ
    for doc in corpus:
        # создается список word_count, длиной равной количеству уникальных слов, и заполняется нулями
        word_count = [0] * len(unique_words)
        # вложенный цикл проходит через каждое слово в документе
        for word in doc.split():
            # увеличивается счетчик для текущего слова в списке word_count. Индекс для увеличения счетчика берется из word_dict
            word_count[word_dict[word]] += 1
        # вектор частот слов, представленный списком word_count, добавляется в bow_corpus
        bow_corpus.append(word_count)
    # возвращает два значения: bow_corpus, который представляет собой список векторов BoW для каждого документа,
    # и word_dict, который представляет собой словарь уникальных слов и их индексов.
    return bow_corpus, word_dict

In [21]:
import math

def tf_idf(corpus):
    # функция bag_of_words применяется к корпусу, возвращая список векторов bag-of-words и словарь, сопоставляющий каждому уникальному слову его индекс
    bow_corpus, word_dict = bag_of_words(corpus)
    # вычисляет количество документов в корпусе
    num_docs = len(corpus)
    # создается пустой список для хранения векторов
    tf_idf_corpus = []
    
    # создается список, заполненный нулями, для подсчета, в скольких документах встречается каждое слово
    word_document_count = [0] * len(word_dict)
    # цикл проходит через все документы и увеличивает счетчик для каждого слова, которое встречается в документе
    for word_count in bow_corpus:
        for i, count in enumerate(word_count):
            if count > 0:
                word_document_count[i] += 1

    # цикл проходит через все документы второй раз
    for word_count in bow_corpus:
        # создается пустой список для хранения вектора TF-IDF для текущего документа
        tf_idf_vector = []
        # цикл проходит через все слова в текущем документе и вычисляет их значения TF и IDF. 
        # значение TF-IDF для каждого слова затем добавляется в вектор TF-IDF для текущего документа
        for i, count in enumerate(word_count):
            # вычисляется значение TF для текущего слова. 
            # TF — это просто отношение количества вхождений слова в документе к общему числу слов в документе.
            tf = count / (sum(word_count) + 1e-10)
            # вычисляется значение IDF для текущего слова. 
            # IDF вычисляется как логарифм отношения общего числа документов к числу документов, содержащих данное слово.
            idf = math.log(num_docs / (1 + word_document_count[i])
            tf_idf_vector.append(tf * idf)
        # вектор TF-IDF для текущего документа добавляется в список всех векторов TF-IDF
        tf_idf_corpus.append(tf_idf_vector)
    # возвращает список всех векторов TF-IDF и словарь, сопоставляющий каждому уникальному слову его индекс
    return tf_idf_corpus, word_dict

In [16]:
X = X.str.lower()

In [17]:
X = X.str.replace('[^\w\s]', '', regex=True)

In [18]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

X = X.apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))

In [20]:
X_bow, word_dictBOW = bag_of_words(X)


In [22]:
X_tf_idf, word_dictTF = tf_idf(X)

In [23]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_bow, y, test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_tf_idf, y, test_size=0.2, random_state=42)

In [25]:
model = LogisticRegression()
model.fit(X_train1, y_train1)
y_pred = model.predict(X_test1)
accuracy = accuracy_score(y_test1, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9787234042553191


In [26]:
model = LogisticRegression()
model.fit(X_train2, y_train2)
y_pred = model.predict(X_test2)
accuracy = accuracy_score(y_test2, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9632495164410058


# Решить задачу классификации с понижением размерности. Использовать самостоятельно реализованные модели из предыдущих ЛР.

In [17]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, RFE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report

In [18]:
def test_class(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
  bag = BaggingClassifier().fit(X_train, y_train)
  print(classification_report(y_test, bag.predict(X_test)))

In [19]:
# 1. SelectKBest
selector = SelectKBest(k=2)  # количество наиболее значимых признаков
X_SelectKBest_classifier1 = selector.fit_transform(X_bow, y)
X_SelectKBest_classifier2 = selector.fit_transform(X_tfidf, y)

In [20]:
test_class(X_SelectKBest_classifier1, y)

              precision    recall  f1-score   support

         ham       0.95      0.96      0.96       917
        spam       0.66      0.60      0.63       117

    accuracy                           0.92      1034
   macro avg       0.80      0.78      0.79      1034
weighted avg       0.92      0.92      0.92      1034



In [21]:
test_class(X_SelectKBest_classifier2, y)

              precision    recall  f1-score   support

         ham       0.91      1.00      0.95       917
        spam       0.90      0.24      0.38       117

    accuracy                           0.91      1034
   macro avg       0.91      0.62      0.67      1034
weighted avg       0.91      0.91      0.89      1034



In [22]:
# 3. RFE (Recursive Feature Elimination)
estimator = BaggingClassifier()
selector = RFE(estimator, n_features_to_select=5)  # количество выбранных признаков (n_features_to_select)
X_RFE_classifier1 = selector.fit_transform(X_SelectKBest_classifier1, y)
X_RFE_classifier2 = selector.fit_transform(X_SelectKBest_classifier2, y)

In [23]:
test_class(X_RFE_classifier1, y)

              precision    recall  f1-score   support

         ham       0.95      0.96      0.96       917
        spam       0.66      0.60      0.63       117

    accuracy                           0.92      1034
   macro avg       0.80      0.78      0.79      1034
weighted avg       0.92      0.92      0.92      1034



In [24]:
test_class(X_RFE_classifier2, y)

              precision    recall  f1-score   support

         ham       0.91      1.00      0.95       917
        spam       0.90      0.24      0.38       117

    accuracy                           0.91      1034
   macro avg       0.91      0.62      0.67      1034
weighted avg       0.91      0.91      0.89      1034



In [25]:
# 6. Isomap
isomap = Isomap(n_components=10)  # количество компонент (n_components)
X_Isomap_classifier1 = isomap.fit_transform(X_bow)
X_Isomap_classifier2 = isomap.fit_transform(X_tfidf)

C:\Users\raevs\Desktop\ML\venv\Lib\site-packages\sklearn\manifold\_isomap.py:383: UserWarning: The number of connected components of the neighbors graph is 6 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
C:\Users\raevs\Desktop\ML\venv\Lib\site-packages\scipy\sparse\_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
C:\Users\raevs\Desktop\ML\venv\Lib\site-packages\sklearn\manifold\_isomap.py:383: UserWarning: The number of connected components of the neighbors graph is 4 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
C:\Users\raevs\Desktop\ML\venv\Lib\site-packages\scipy\sparse\_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is mor

In [26]:
test_class(X_Isomap_classifier1, y)

              precision    recall  f1-score   support

         ham       0.94      0.97      0.95       917
        spam       0.69      0.52      0.59       117

    accuracy                           0.92      1034
   macro avg       0.81      0.75      0.77      1034
weighted avg       0.91      0.92      0.91      1034



In [27]:
test_class(X_Isomap_classifier2, y)

              precision    recall  f1-score   support

         ham       0.93      0.97      0.95       917
        spam       0.62      0.41      0.49       117

    accuracy                           0.90      1034
   macro avg       0.77      0.69      0.72      1034
weighted avg       0.89      0.90      0.90      1034



# Решить задачу тематического моделирования с помощью LDA.

In [38]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\raevs\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [43]:
# Создание объекта CountVectorizer для извлечения признаков
vectorizer = CountVectorizer(max_features=1000,   # Ограничение на максимальное количество признаков
                             stop_words='english',  # Игнорирование стоп-слов
                             max_df=0.5,  # Игнорирование терминов, которые появляются в более чем 50% документов
                             min_df=2)  # Игнорирование терминов, которые появляются в менее чем 2 документах


In [44]:
# Преобразование текстовых данных в матрицу признаков
X_features = vectorizer.fit_transform(X)


In [45]:
# Создание модели LDA
num_topics = 10  # Количество тем
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)

In [46]:
# Обучение модели LDA
lda_model.fit(X_features)

# Получение самых вероятных слов для каждой темы
feature_names = vectorizer.get_feature_names_out()
top_words = 10  # Количество верхних слов для вывода

In [47]:
for topic_idx, topic in enumerate(lda_model.components_):
    top_feature_indices = topic.argsort()[:-top_words - 1:-1]
    top_features = [feature_names[i] for i in top_feature_indices]
    print(f"Тема #{topic_idx + 1}:")
    print(top_features)
    print()

Тема #1:
['know', 'did', 'right', 'yes', 'just', 'tonight', 'say', 'gonna', 'phone', 'like']

Тема #2:
['lor', 'got', 'ok', 'oh', 'wat', 'pls', 'dun', 'wan', 'ask', 'ìï']

Тема #3:
['tell', 'just', 'don', 'yeah', 'sure', 'time', 'got', 'ok', 'going', 'll']

Тема #4:
['like', 'going', 'da', 'way', 'feel', 'think', 'time', 'ya', 'life', 'make']

Тема #5:
['claim', 'prize', 'won', 'number', 'cash', 'www', 'urgent', 'win', 'com', 'txt']

Тема #6:
['good', 'love', 'day', 'ì_', 'hi', 'hope', 'happy', 'morning', 'babe', 'miss']

Тема #7:
['ll', 'just', 'sorry', 'need', 'want', 'come', 'sent', 'dont', 'later', 'time']

Тема #8:
['txt', 'home', 'new', 'ur', 'chat', '150p', 'send', 'stop', 'week', 'free']

Тема #9:
['gt', 'lt', 'ur', 'ok', 'reply', 'msg', 'send', 'sms', 'heart', 'text']

Тема #10:
['free', 'text', 'know', 'mobile', 'stop', 'let', 'phone', 'reply', 'aight', 'txt']



In [48]:
# Получение распределения тем для каждого документа
topic_distribution = lda_model.transform(X_features)

In [50]:
most_likely_topics = topic_distribution.argmax(axis=1)

In [51]:
most_likely_topics

array([0, 3, 4, ..., 0, 0, 6], dtype=int64)

In [52]:
df['most_likely_topic'] = most_likely_topics

In [53]:
df

,v1,v2,most_likely_topic
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,3
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,4
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5164,spam,This is the 2nd time we have tried 2 contact u...,4
5165,ham,Will Ì_ b going to esplanade fr home?,5
5166,ham,"Pity, * was in mood for that. So...any other s...",0
5167,ham,The guy did some bitching but I acted like i'd...,0
